In [174]:
import pandas  as pd

In [175]:
data = pd.read_csv('data.csv')

In [176]:
data.head()

,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,sqft_above,sqft_basement,yr_built,yr_renovated,street,city,statezip,country
0,2014-05-02 00:00:00,313000.0,3.0,1.50,1340,7912,1.5,0,0,3,1340,0,1955,2005,18810 Densmore Ave N,Shoreline,WA 98133,USA
1,2014-05-02 00:00:00,2384000.0,5.0,2.50,3650,9050,2.0,0,4,5,3370,280,1921,0,709 W Blaine St,Seattle,WA 98119,USA
2,2014-05-02 00:00:00,342000.0,3.0,2.00,1930,11947,1.0,0,0,4,1930,0,1966,0,26206-26214 143rd Ave SE,Kent,WA 98042,USA
3,2014-05-02 00:00:00,420000.0,3.0,2.25,2000,8030,1.0,0,0,4,1000,1000,1963,0,857 170th Pl NE,Bellevue,WA 98008,USA
4,2014-05-02 00:00:00,550000.0,4.0,2.50,1940,10500,1.0,0,0,4,1140,800,1976,1992,9105 170th Ave NE,Redmond,WA 98052,USA


In [177]:
data.isna().sum()

date             0
price            0
bedrooms         0
bathrooms        0
sqft_living      0
sqft_lot         0
floors           0
waterfront       0
view             0
condition        0
sqft_above       0
sqft_basement    0
yr_built         0
yr_renovated     0
street           0
city             0
statezip         0
country          0
dtype: int64

In [178]:
data.nunique()

date               70
price            1741
bedrooms           10
bathrooms          26
sqft_living       566
sqft_lot         3113
floors              6
waterfront          2
view                5
condition           5
sqft_above        511
sqft_basement     207
yr_built          115
yr_renovated       60
street           4525
city               44
statezip           77
country             1
dtype: int64

In [179]:
data.nunique().index

Index(['date', 'price', 'bedrooms', 'bathrooms', 'sqft_living', 'sqft_lot',
       'floors', 'waterfront', 'view', 'condition', 'sqft_above',
       'sqft_basement', 'yr_built', 'yr_renovated', 'street', 'city',
       'statezip', 'country'],
      dtype='object')

In [180]:
data.drop('country', axis=1, inplace=True)

In [181]:
data[data['price'] <= 0]['price'].unique()

array([0.])

In [182]:
def remove_casa_free(df):
    index = df[df['price'] <= 0].index
    return df.drop(index=index)

In [183]:
data = remove_casa_free(data)

In [184]:
import tensorflow as tf
import keras

In [185]:
def tokenizer_text(data):
    vectorizer = keras.layers.TextVectorization(
        output_mode='int',
        output_sequence_length=10,
        standardize='lower_and_strip_punctuation',
    )
    # unindo colunas de
    data['location'] = data['street'] + ' ' + data['city'] + ' ' + data['statezip'] 

    # vetorizando
    vectorizer.adapt(data['location'].values)
    tokenizer_text = vectorizer(data['location'].values)

    return tokenizer_text

In [186]:
subset = tokenizer_text(data)

In [187]:
def to_colunmns(data, subset):
    for i in range(subset.shape[1]):
        data[f'char_token_{i}'] = subset[:, i]

    return data

In [188]:
def remove_columns(df, columns_to_remove):
    return df.drop(columns_to_remove, axis=1)

In [189]:
data = remove_columns(data, ['street', 'city', 'statezip', 'location'])

In [191]:
def arrendondamento(data):
    data['bathrooms'] = data['bathrooms'].astype(int)
    return data

In [192]:
data = padronizacao(data)

In [193]:
data = to_colunmns(data, subset)

In [194]:
def dummies(data):
    return pd.get_dummies(data=data, columns=['floors', 'waterfront', 'view', 'condition'], dtype=int)

In [195]:
data = dummies(data)

In [196]:
data = arrendondamento(data)

In [197]:
import numpy as np
def transformacao_log(data):
    data['price'] = np.log1p(data['price'])
    return data

In [198]:
data = transformacao_log(data)

In [199]:
X = data.drop(['price','date'], axis=1).values

In [200]:
y = data['price'].values

In [201]:
data

,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,sqft_above,sqft_basement,yr_built,yr_renovated,...,view_0,view_1,view_2,view_3,view_4,condition_1,condition_2,condition_3,condition_4,condition_5
0,2014-05-02 00:00:00,12.653962,3.0,1,-0.828976,-0.192527,-0.564425,-0.671413,1955,2005,...,1,0,0,0,0,0,0,1,0,0
1,2014-05-02 00:00:00,14.684291,5.0,2,1.587735,-0.160880,1.811625,-0.065270,1921,0,...,0,0,0,0,1,0,0,0,0,1
2,2014-05-02 00:00:00,12.742569,3.0,2,-0.211721,-0.080319,0.126151,-0.671413,1966,0,...,1,0,0,0,0,0,0,0,1,0
3,2014-05-02 00:00:00,12.948012,3.0,2,-0.138487,-0.189245,-0.962384,1.493385,1963,0,...,1,0,0,0,0,0,0,0,1,0
4,2014-05-02 00:00:00,13.217675,4.0,2,-0.201259,-0.120558,-0.798519,1.060425,1976,1992,...,1,0,0,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4595,2014-07-09 00:00:00,12.638399,3.0,1,-0.651123,-0.235686,-0.365446,-0.671413,1954,1979,...,1,0,0,0,0,0,0,0,1,0
4596,2014-07-09 00:00:00,13.188777,3.0,2,-0.703433,-0.201954,-0.423969,-0.671413,1983,2009,...,1,0,0,0,0,0,0,1,0,0
4597,2014-07-09 00:00:00,12.940614,3.0,2,0.918170,-0.217499,1.390257,-0.671413,2009,0,...,1,0,0,0,0,0,0,1,0,0
4598,2014-07-10 00:00:00,12.222935,4.0,2,-0.044330,-0.228177,-0.880452,1.536681,1974,0,...,1,0,0,0,0,0,0,1,0,0


### modelo 1

In [202]:
from sklearn.ensemble import RandomForestRegressor

In [203]:
rf = RandomForestRegressor()
rf.fit(X, y)

RandomForestRegressor()

In [224]:
from sklearn.metrics import mean_squared_error, r2_score

In [223]:
mean_squared_error(y, rf.predict(X))

0.011903041071191044

In [222]:
r2_score(y, rf.predict(X))

0.9595896133198019

In [208]:
from sklearn.linear_model import SGDRegressor

In [209]:
sgd = SGDRegressor()
sgd.fit(X, y)

SGDRegressor()

In [210]:
mean_squared_error(y, sgd.predict(X))

3.035799424111879e+30

In [211]:
r2_score(y, sgd.predict(X))

-1.0306427397684226e+31

In [218]:
from sklearn.svm import SVR

In [219]:
svr = SVR()
svr.fit(X, y)

SVR()

In [220]:
svr.score(X, y) 

0.035543996224436714

In [221]:
r2_score(y, svr.predict(X))

0.035543996224436714

In [230]:
data.drop(['date','price'], axis=1)

,bedrooms,bathrooms,sqft_living,sqft_lot,sqft_above,sqft_basement,yr_built,yr_renovated,char_token_0,char_token_1,...,view_0,view_1,view_2,view_3,view_4,condition_1,condition_2,condition_3,condition_4,condition_5
0,3.0,1,-0.828976,-0.192527,-0.564425,-0.671413,1955,2005,1126,333,...,1,0,0,0,0,0,0,1,0,0
1,5.0,2,1.587735,-0.160880,1.811625,-0.065270,1921,0,576,25,...,0,0,0,0,1,0,0,0,0,1
2,3.0,2,-0.211721,-0.080319,0.126151,-0.671413,1966,0,3199,223,...,1,0,0,0,0,0,0,0,1,0
3,3.0,2,-0.138487,-0.189245,-0.962384,1.493385,1963,0,1792,195,...,1,0,0,0,0,0,0,0,1,0
4,4.0,2,-0.201259,-0.120558,-0.798519,1.060425,1976,1992,1727,195,...,1,0,0,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4595,3.0,1,-0.651123,-0.235686,-0.365446,-0.671413,1954,1979,920,13,...,1,0,0,0,0,0,0,0,1,0
4596,3.0,2,-0.703433,-0.201954,-0.423969,-0.671413,1983,2009,4232,6,...,1,0,0,0,0,0,0,1,0,0
4597,3.0,2,0.918170,-0.217499,1.390257,-0.671413,2009,0,1945,533,...,1,0,0,0,0,0,0,1,0,0
4598,4.0,2,-0.044330,-0.228177,-0.880452,1.536681,1974,0,2313,9,...,1,0,0,0,0,0,0,1,0,0


In [ ]:
def model_deepside(vocab_size, input_shape):
    # Rede do token
    input_token = keras.Input(shape=(60,), name='input_token')
    camada_1_token = keras.layers.Embedding(input_dim=vocab_size, output_dim=16)(input_token)
    camada_2_token = keras.layers.GlobalAveragePooling1D()(camada_1_token)
    camada_3_token = keras.layers.Dense(64, activation='relu')(camada_2_token)
    camada_4_token = keras.layers.Dense(30)(camada_3_token)

    input_normal = keras.layers.Input(shape=input_shape, name='Rede normal')
    camada_2 = keras.layers.Dense(40, activation=keras.activations.selu, kernel_initializer=keras.initializers.lecun_normal)(input_normal)
    camada_3 = keras.layers.Dense(20, activation=keras.activations.selu, kernel_initializer=keras.initializers.lecun_normal)(camada_2)
    camada_4 = keras.layers.Dropout(rate=0.2)(camada_3)
    camada_5 = keras.layers.Dense(10, activation=keras.activations.selu, kernel_initializer=keras.initializers.lecun_normal)(camada_4)
    camada_6 = keras.layers.Dropout(rate=0.2)(camada_5)
    camada_7 = keras.layers.Dense(5, activation=keras.activations.selu, kernel_initializer=keras.initializers.lecun_normal)(camada_6)
    camada_8 = keras.layers.Dropout(rate=0.2)(camada_7)


    concat = keras.layers.concatenate([camada_4_token, camada_8])
    saida  = keras.layers.Dense(1)(concat)
    return keras.Model(inputs=[input_token, input_normal], outputs=saida)